In [10]:
%load_ext autoreload
%autoreload 2
import mmcv
from mmdet3d.apis import inference_mono_3d_detector, init_model
from mmdet3d.registry import VISUALIZERS
from mmengine import load, dump
from mmdet3d.registry import VISUALIZERS
import cv2
import numpy as np
import sys

sys.path.append('../tools')

from data_processor import DataProcessor
from visualizer import ImageVisualizer, draw_monodetection_labels, draw_monodetection_results

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
file = '002238'
dataset = 'dair'
if dataset == 'kitti':
    img_file = '/home/javier/datasets/kitti/training/image_2/' + file + '.png'
    labels_file = '/home/javier/datasets/kitti/training/label_2/'+ file + '.txt'
    calib_file = '/home/javier/datasets/kitti/training/calib/'+ file + '.txt'
    pitch = 0.0
elif dataset == 'dair':
    img_file = '/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/image_2/' + file + '.png'
    labels_file = '/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/label_2/'+ file + '.txt'
    calib_file = '/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/calib/'+ file + '.txt'
    pitch = 0.2031

ann_file = '/home/javier/sensus-loci/sensus/notebooks/tmp/data/dair/002238.pkl'
cam_type = 'CAM_BACK'

## Draw Labels

In [7]:
draw_monodetection_labels(img_file, calib_file, labels_file, num_cars=15, pitch=pitch, thickness=1)

[1.48235, 1.937682, 4.241219]
[1.712159, 2.069001, 4.347082]
[1.795317, 2.09444, 4.311423]
[1.403052, 2.113689, 4.268273]
[1.456858, 1.816689, 4.416424]
[3.19582, 3.196433, 8.162645]
[1.910056, 2.262334, 4.455209]
[1.645071, 2.11366, 4.337559]
[1.340647, 2.002429, 4.191494]
[3.713664, 3.73082, 7.499055]
[1.576959, 1.815553, 4.189867]
[1.421129, 2.02112, 4.740007]
[1.090961, 2.16069, 4.240069]


## Inference

In [11]:
config_fcos = '/home/javier/sensus-loci/sensus/configs/smoke/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d.py'

checkpoint_fcos = '/home/javier/sensus-loci/work_dirs/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d/epoch_100.pth'

device = 'cuda:0'
model_mmdet = init_model(config_fcos, checkpoint_fcos, device=device)
result = inference_mono_3d_detector(model_mmdet, img_file, ann_file, cam_type)

# init visualizer
visualizer_mmdet = VISUALIZERS.build(model_mmdet.cfg.visualizer)
visualizer_mmdet.dataset_meta = model_mmdet.dataset_meta

Loads checkpoint by local backend from path: /home/javier/sensus-loci/work_dirs/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d/epoch_100.pth
07/31 14:07:08 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_0.projs.0.conv is upgraded to version 2.
07/31 14:07:08 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_0.nodes.0.conv is upgraded to version 2.
07/31 14:07:08 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_1.projs.0.conv is upgraded to version 2.
07/31 14:07:08 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_1.projs.1.conv is upgraded to version 2.
07/31 14:07:08 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_1.nodes.0.conv is upgraded to version 2.
07/31 14:07:08 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_1.nodes.1.conv is upgraded to version 2.
07/31 14:07:08 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_2.projs.0.conv is upgraded to version 2.
07/31 14:07:08 - mmengine - INFO - ModulatedDef

In [12]:
print(result.pred_instances_3d.scores_3d)

tensor([0.5217, 0.4985, 0.4795, 0.4576, 0.4507, 0.4311, 0.4160, 0.4013, 0.3731,
        0.3315, 0.3309, 0.3294, 0.3123, 0.3022], device='cuda:0')


## MMDet Viz

In [13]:
img = mmcv.imread(img_file)
img = mmcv.imconvert(img, 'bgr', 'rgb')
data_input = dict(img=img)
# show the results
visualizer_mmdet.add_datasample(
    'result',
    data_input,
    data_sample=result,
    draw_gt=False,
    show=True,
    wait_time=0,
    out_file='result_mmdet',
    pred_score_thr= 0.30,
    vis_task='mono_det')

/home/javier/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:812: UserWarning: Warning: The polygon is out of bounds, the drawn polygon may not be in the image
  warnings.warn(


## Sensus Viz

In [19]:
draw_monodetection_results(img_file, calib_file, result, score=0.40, pitch=pitch, thickness=1)

[1.542670726776123, 1.8651961088180542, 4.262558460235596]
[1.5057939291000366, 1.856995701789856, 4.242576599121094]
[1.4859989881515503, 1.8845899105072021, 4.385947227478027]
[2.678711414337158, 2.51883602142334, 6.373397350311279]
[1.727476954460144, 1.9132280349731445, 4.239100456237793]
[1.7327828407287598, 1.9342377185821533, 4.45134162902832]
[1.8245452642440796, 1.9512022733688354, 4.261423587799072]
[1.4941898584365845, 1.8839384317398071, 4.270991802215576]
